<a href="https://colab.research.google.com/github/hybrits/H-ALLY/blob/main/Primer_entramientoV_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast

# Cargar el archivo CSV
df = pd.read_csv('/content/foro_data_paginas_limpio_ingles (1).csv')

# Crear un mapeo de etiquetas único para cada valor en la columna 'Título'
label_mapping = {label: idx for idx, label in enumerate(df['Título'].unique())}

# Aplicar el mapeo de etiquetas a la columna 'Título'
df['Título'] = df['Título'].map(label_mapping)

# Verificar los valores NaN en 'Título'
nan_titles = df['Título'].isna()
print("Filas con NaN en 'Título':")
print(df[nan_titles])

# Manejar valores NaN: Opción 1 - Eliminar filas con NaN
df = df.dropna(subset=['Título'])

# Convertir 'Título' a enteros después de manejar NaN
df['Título'] = df['Título'].astype(int)

# Verificar que no queden NaN en 'Título'
print("Número de valores NaN en 'Título' después de la limpieza:", df['Título'].isna().sum())

# Asegurarse de que todos los valores en 'Contenido' sean cadenas de texto
df['Contenido'] = df['Contenido'].fillna('').astype(str)

# Tokenización usando DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(df['Contenido'].tolist(), truncation=True, padding=True, max_length=512)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(train_encodings['input_ids'], df['Título'], test_size=0.2, random_state=42)

print("Datos preparados para el entrenamiento.")


Filas con NaN en 'Título':
Empty DataFrame
Columns: [Título, Contenido]
Index: []
Número de valores NaN en 'Título' después de la limpieza: 0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Datos preparados para el entrenamiento.


In [2]:
import torch
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Verificar si hay una GPU disponible y configurarla para el entrenamiento
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Crear el modelo
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_mapping))
model.to(device)

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',          # Directorio de salida para guardar los resultados
    num_train_epochs=3,              # Número de épocas de entrenamiento
    per_device_train_batch_size=8,   # Tamaño del lote por dispositivo durante el entrenamiento
    per_device_eval_batch_size=8,    # Tamaño del lote por dispositivo durante la evaluación
    warmup_steps=500,                # Número de pasos para el calentamiento
    weight_decay=0.01,               # Tasa de decaimiento del peso
    logging_dir='./logs',            # Directorio de registro para TensorBoard
    logging_steps=10,
    eval_strategy="epoch"            # Evaluación al final de cada época
)

# Convertir las entradas y etiquetas de entrenamiento y prueba a tensores de PyTorch
X_train = torch.tensor(X_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.long)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

# Crear un dataset a partir de las entradas y etiquetas
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Crear datasets de entrenamiento y prueba
train_dataset = CustomDataset({'input_ids': X_train, 'attention_mask': train_encodings['attention_mask'][:len(X_train)]}, y_train)
test_dataset = CustomDataset({'input_ids': X_test, 'attention_mask': train_encodings['attention_mask'][len(X_train):]}, y_test)

# Crear el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Entrenar el modelo
trainer.train()

# Evaluar el modelo
trainer.evaluate()


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-2-6c85cb76e38f>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-2-6c85cb76e38f>:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Epoch,Training Loss,Validation Loss
1,5.738600,5.718251
2,5.485300,5.638207
3,5.305300,5.551589


<ipython-input-2-6c85cb76e38f>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-2-6c85cb76e38f>:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
<ipython-input-2-6c85cb76e38f>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-2-6c85cb76e38f>:38: UserWarning: To copy construct from a tensor, it is recommended to

{'eval_loss': 5.551588535308838,
 'eval_runtime': 12.1038,
 'eval_samples_per_second': 55.602,
 'eval_steps_per_second': 7.023,
 'epoch': 3.0}